In [1]:
import rl
import utils
import custom_model
from stable_baselines3.common.torch_layers import NatureCNN

In [27]:
screen_size=42

vec_env = rl.setup_vec_env(
        n_envs=5,
        frame_skip=4,
        screen_size=screen_size,
        action_repeat_probability=0.0,
        frame_stacks=4,
        seed=0,
)

In [3]:
model = rl.load(path="C:/Users/cgoet/PycharmProjects/Pong-RL/models/ppo_small_v1", vec_env=vec_env)

In [25]:
model = rl.setup_model(
        vec_env=vec_env,
        model_class=NatureCNN,
        model_kwargs={"features_dim": 256},
        device="cuda",
        log_dir="C:/Users/cgoet/PycharmProjects/Pong-RL/logs/",
)

In [4]:
utils.print_model_parameters(model)

features_extractor: 94,624
pi_features_extractor: 94,624
vf_features_extractor: 94,624
mlp_extractor: 32,896
action_net: 390
value_net: 65
Total number of parameters: 317,223


In [ ]:
# tensorboard --logdir="C:/Users/cgoet/PycharmProjects/Pong-RL/logs"

In [17]:
model = rl.train(model, steps=120_000, name="ppo_large_vec", new_run=True)  # start new training run

In [17]:
model = rl.train(model, steps=700_000, name="ppo_small", new_run=False)  # continue training run

In [18]:
rl.save(model, path="C:/Users/cgoet/PycharmProjects/Pong-RL/models/ppo_small_v1")

In [19]:
mean_reward, std_reward = rl.evaluate(model, vec_env, episodes=10, deterministic=False)

Mean reward: -13.90 +/- 1.51


In [14]:
rewards = rl.visualize(model=None, screen_size=screen_size, episodes=1)  # random agent

Episode 0 reward: -20.0
Average reward: -20.0


In [13]:
rewards = rl.visualize(model, screen_size=screen_size, episodes=1)  # trained agent

Episode 0 reward: -21.0
Average reward: -21.0


In [20]:
from time import sleep

# test model fast
test_env = rl.setup_vec_env(
        n_envs=1,
        frame_skip=4,
        screen_size=screen_size,
        action_repeat_probability=0.0,
        frame_stacks=4,
        seed=0,
)
delay = 1 / 10

episodes = 1
episode_rewards = []
for episode in range(episodes):
    # obs shape (n_envs, 84, 84, n_frame_stacks)
    obs = test_env.reset()
    dones = False
    total_reward = 0

    while not dones:
        action, _states = model.predict(obs, deterministic=False)
        obs, reward, dones, info = test_env.step(action)
        total_reward += reward[0]
        test_env.render("human")
        sleep(delay)
        
    episode_rewards.append(total_reward)
    print(f"Episode {episode} reward: {total_reward}")

C:\Users\cgoet\PycharmProjects\Pong-RL\.venv\lib\site-packages\gymnasium\utils\passive_env_checker.py:335: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Episode 0 reward: -15.0
